# December 17, 2023

https://adventofcode.com/2023/day/17

In [3]:
from queue import PriorityQueue
from collections import defaultdict

In [5]:
tmp = defaultdict( list )

In [24]:
test = f'''2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533'''

test = test.split("\n")
test = [ [int(y) for y in line] for line in test]

In [27]:
fn = "data/17.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz = [x.strip() for x in text]
puzz = [ [int(y) for y in line] for line in puzz ]

### Part 1

In [78]:
# row and col refer to map[row][col] in the text
# hist is up to three letters all the same, such as "uuu" or "rr"
def encode( row, col, hist ):
    return str(row) + ":" + str(col) + ":" + hist

def split_label( label ):
    # split out hist, but leave row:col together
    row, col, hist = label.split(":")
    return row + ":" + col, hist

def decode( label ):
    row, col, hist = label.split(":")
    return int(row), int(col), hist

def update_hist( hist, dir ):
    # update history after moving in dir 
    # basically appending it if we haven't changed direction, else overwriting it
    # returns None if it makes the hist too long in one direction or we try to backtrack

    # PART 1:
    #min_straight=0
    #max_straight=3

    # PART 2:
    min_straight=4
    max_straight=10

    # X is for initial case where we haven't moved at all yet
    no_backsies = {'u':'d', 'd':'u', 'l':'r', 'r':'l', "X":"X"}

    if dir == no_backsies[hist[-1]]:
        return None
    
    if dir == hist[0]:
        # check if we can take another step this direction
        if len(hist) == max_straight:
            return None
        return hist + dir
    else:
        # check if we can turn yet
        if (len(hist) < min_straight) and hist[0] != "X":
            return None
        return dir
    
def find_neighbors( row, col, hist, mapa ):
    nbrs = list()
    if row > 0:
        hist2 = update_hist(hist, "u")
        if hist2 is not None:
            nbrs.append( encode(row-1, col, hist2) )

    if row < len(mapa)-1:
        hist2 = update_hist(hist, "d")
        if hist2 is not None:
            nbrs.append( encode(row+1, col, hist2) )

    if col > 0:
        hist2 = update_hist(hist, "l")
        if hist2 is not None:
            nbrs.append( encode(row, col-1, hist2) )

    if col < len(mapa[0])-1:
        hist2 = update_hist(hist, "r")
        if hist2 is not None:
            nbrs.append( encode(row, col+1, hist2) )

    return nbrs

def approx( row, col, goal, min_cost = 1 ):
    #r,c, _ = decode(nbr)
    return ((goal[0] - row) + (goal[1]-col)) * min_cost
    
def assemble_path( cur, came_from ):
    path = [cur]
    while came_from[ path[0] ] is not None:
        path.insert(0, came_from[path[0]] )

    return path

def astar( mapa, start=[0,0], goal=None ):
    if goal is None:
        goal = [len(mapa)-1, len(mapa[0])-1]

    frontier = PriorityQueue()
    explored = dict() # keyed by state, valued by cost from start to that state
    came_from = dict()

    # initial status
    start_lab = encode(0,0, "X")
    frontier.put( (approx(start[0], start[1], goal), start_lab ) )
    explored[start_lab] = 0
    came_from[start_lab] = None

    while not frontier.empty():
        cur = frontier.get()[1] # 0 is the priority
        row, col, hist = decode(cur)
        if [row, col] == goal:
            break

        cost_so_far = explored[cur]
        nbrs = find_neighbors( row, col, hist, mapa )
       # print(cur)
        #print(nbrs)
        for n in nbrs:
            r,c, hist = decode( n )
            new_cost = cost_so_far + mapa[r][c]

            if n not in explored or new_cost < explored[n]:
                explored[n] = new_cost
                priority = new_cost + approx( r, c, goal )
                frontier.put( (priority, n) )
                came_from[n] = cur

    if [row, col] == goal:
        print("Yes!")
        path = assemble_path( cur, came_from )
        return explored[cur], path, came_from







In [79]:
cost, pth, came_from = astar( test )

Yes!


In [80]:
cost, pth

(94,
 ['0:0:X',
  '0:1:r',
  '0:2:rr',
  '0:3:rrr',
  '0:4:rrrr',
  '0:5:rrrrr',
  '0:6:rrrrrr',
  '0:7:rrrrrrr',
  '0:8:rrrrrrrr',
  '1:8:d',
  '2:8:dd',
  '3:8:ddd',
  '4:8:dddd',
  '4:9:r',
  '4:10:rr',
  '4:11:rrr',
  '4:12:rrrr',
  '5:12:d',
  '6:12:dd',
  '7:12:ddd',
  '8:12:dddd',
  '9:12:ddddd',
  '10:12:dddddd',
  '11:12:ddddddd',
  '12:12:dddddddd'])

In [67]:
cost, pth, came_from = astar( puzz )

Yes!


In [68]:
cost

797

### Part 2

In [77]:
cost, pth, came_from = astar( test )

0:0:X
['1:0:d', '0:1:r']
1:0:d
['2:0:dd']
0:1:r
['0:2:rr']
0:2:rr
['0:3:rrr']
2:0:dd
['3:0:ddd']
0:3:rrr
['0:4:rrrr']
3:0:ddd
['4:0:dddd']
0:4:rrrr
['1:4:d', '0:5:rrrrr']
4:0:dddd
['5:0:ddddd', '4:1:r']
5:0:ddddd
['6:0:dddddd', '5:1:r']
0:5:rrrrr
['1:5:d', '0:6:rrrrrr']
0:6:rrrrrr
['1:6:d', '0:7:rrrrrrr']
1:4:d
['2:4:dd']
2:4:dd
['3:4:ddd']
5:1:r
['5:2:rr']
6:0:dddddd
['7:0:ddddddd', '6:1:r']
0:7:rrrrrrr
['1:7:d', '0:8:rrrrrrrr']
0:8:rrrrrrrr
['1:8:d', '0:9:rrrrrrrrr']
0:9:rrrrrrrrr
['1:9:d', '0:10:rrrrrrrrrr']
1:6:d
['2:6:dd']
4:1:r
['4:2:rr']
1:5:d
['2:5:dd']
5:2:rr
['5:3:rrr']
7:0:ddddddd
['8:0:dddddddd', '7:1:r']
0:10:rrrrrrrrrr
['1:10:d']
1:8:d
['2:8:dd']
6:1:r
['6:2:rr']
3:4:ddd
['4:4:dddd']
4:2:rr
['4:3:rrr']
1:7:d
['2:7:dd']
1:9:d
['2:9:dd']
2:5:dd
['3:5:ddd']
2:6:dd
['3:6:ddd']
8:0:dddddddd
['9:0:ddddddddd', '8:1:r']
2:8:dd
['3:8:ddd']
6:2:rr
['6:3:rrr']
7:1:r
['7:2:rr']
1:10:d
['2:10:dd']
2:9:dd
['3:9:ddd']
9:0:ddddddddd
['10:0:dddddddddd', '9:1:r']
10:0:dddddddddd
['10:1:r']

In [ ]:
cost, pth

(102,
 ['0:0:X',
  '0:1:r',
  '0:2:rr',
  '1:2:d',
  '1:3:r',
  '1:4:rr',
  '1:5:rrr',
  '0:5:u',
  '0:6:r',
  '0:7:rr',
  '0:8:rrr',
  '1:8:d',
  '1:9:r',
  '2:9:d',
  '2:10:r',
  '3:10:d',
  '4:10:dd',
  '4:11:r',
  '5:11:d',
  '6:11:dd',
  '7:11:ddd',
  '7:12:r',
  '8:12:d',
  '9:12:dd',
  '10:12:ddd',
  '10:11:l',
  '11:11:d',
  '12:11:dd',
  '12:12:r'])

In [81]:
cost, pth, came_from = astar( puzz )

Yes!


In [82]:
cost

914